In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os

from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [6]:
data = pd.read_csv('/path/to/text/data')

In [7]:
nlp = spacy.load('en_core_web_lg')

In [9]:
data = data.drop(columns=['Unnamed: 0'])

In [10]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [11]:
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [14]:
tqdm.pandas()
data["processed"] = data["impact details"].progress_apply(spacy_tokenizer)

100%|██████████| 1150/1150 [01:31<00:00, 12.52it/s]


In [1]:
#doc = nlp(data["processed"][0])
doc = nlp(data["impact details"][0])
#spacy.displacy.render(doc, style='ent',jupyter=True)

NameError: name 'nlp' is not defined

In [26]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data["processed"])

In [27]:
NUM_TOPICS = 10

In [28]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=5, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [29]:
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [30]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('child', 1859.7699375375396), ('policy', 1840.3355979514745), ('use', 1175.8412844276434), ('service', 1110.6109458062242), ('government', 1099.2238157460715), ('report', 1078.368015734531), ('national', 1060.076723919624), ('evidence', 1042.2564111812203), ('programme', 978.8130108176606), ('education', 973.7971159527904)]
Topic 1:
[('injury', 64.49920053146022), ('safety', 63.64308627319845), ('poetry', 53.05493536039165), ('road', 49.58799740293328), ('paralympic', 43.406566597615154), ('athlete', 40.97254650853041), ('speed', 40.54432095092614), ('asylum', 34.41292658576217), ('transport', 33.83700080036442), ('death', 32.617202710002914)]
Topic 2:
[('teacher', 1107.0541410797728), ('education', 923.8503333913324), ('school', 740.1082542526997), ('student', 578.1321280851536), ('teach', 533.192983603844), ('curriculum', 362.9640757230143), ('university', 349.88751572641513), ('learn', 344.8765344266833), ('practice', 247.63479688726363), ('use', 236.9523418156

In [32]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      116.414551  -18.453203       1        1  23.152124
3     -104.846588  -24.452402       2        1  19.352607
7       73.571754   69.631104       3        1  18.421929
5      -83.452164  151.204208       4        1  18.145774
9        5.292179  -17.097040       5        1  10.862536
2      -36.564137   62.278008       6        1   5.339133
6      -56.351395 -109.557762       7        1   3.641116
4     -147.680939   63.638065       8        1   0.401483
1       52.187092 -106.019028       9        1   0.365552
8       25.085737  154.739410      10        1   0.317747, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
2397  Default  1721.000000         health  1721.000000  30.0000  30.0000
1699  Default  2380.000000      education  2380.000000  29.0000  29.0000
5184  Default  1385.000000        teacher  1385.000000  28.0000  28.0000
4656  Default  2079.000000         school  2079.000000  27.0000  27.0000
2970  Default  1389.000000            law  1389.000000  26.0000  26.0000
832   Default  1858.000000          child  1858.000000  25.0000  25.0000
5032  Default   935.000000        student   935.000000  24.0000  24.0000
5513  Default  3475.000000            use  3475.000000  23.0000  23.0000
3370  Default   687.000000          model   687.000000  22.0000  22.0000
978   Default  1884.000000     commission  1884.000000  21.0000  21.0000
5183  Default   658.000000          teach   658.000000  20.0000  20.0000
2896  Default  1019.000000        justice  1019.000000  19.0000  19.0000
1241  Default   703.000000          court   703.000000  18.0000  18.0000
4198  Default  2651.000000         public  2651.000000  17.0000  17.0000
2988  Default   842.000000          learn   842.000000  16.0000  16.0000
3004  Default   875.000000          legal   875.000000  15.0000  15.0000
267   Default   291.000000           anti   291.000000  14.0000  14.0000
5479  Default  1486.000000     university  1486.000000  13.0000  13.0000
838   Default   243.000000          china   243.000000  12.0000  12.0000
1892  Default  1316.000000       european  1316.000000  11.0000  11.0000
724   Default   917.000000           care   917.000000  10.0000  10.0000
3995  Default  2275.000000       practice  2275.000000   9.0000   9.0000
1302  Default   482.000000     curriculum   482.000000   8.0000   8.0000
3945  Default  5680.000000         policy  5680.000000   7.0000   7.0000
4926  Default   810.000000          sport   810.000000   6.0000   6.0000
2500  Default   897.000000          human   897.000000   5.0000   5.0000
321   Default   891.000000           area   891.000000   4.0000   4.0000
738   Default  1112.000000           case  1112.000000   3.0000   3.0000
344   Default   720.000000        article   720.000000   2.0000   2.0000
2762  Default  2397.000000  international  2397.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
1722  Topic10     3.186095         elders     9.404032   4.6693  -6.2915
4340  Topic10     5.149854   recreational    17.761582   4.5136  -5.8113
4738  Topic10     1.854285       servants     6.455696   4.5042  -6.8327
2239  Topic10     2.242672   generational     7.883380   4.4946  -6.6426
5436  Topic10     3.961250       uncitral    15.144974   4.4106  -6.0737
3833  Topic10    13.829437        penalty    52.996301   4.4082  -4.8234
2643  Topic10     1.310132     inevitable     5.028011   4.4068  -7.1801
1019  Topic10     2.344941     competitor     9.319804   4.3718  -6.5980
4166  Topic10     1.751511     protective     7.122714   4.3489  -6.8898
5173  Topic10     2.339302            tap     9.602840   4.3395  -6.6004
4316  Topic10     1.404549      receptive     5.941433   4.3094  -7.1105
4875  Topic10     3.759807           song    16.0

In [33]:
text = spacy_tokenizer('policy')
print(lda.transform(vectorizer.transform([text])))

[[0.05001679 0.05000128 0.05000177 0.54995067 0.05000026 0.05000851
  0.05000849 0.05000535 0.05000145 0.05000545]]
